In [114]:
# Import all modules that are required
import numpy as np
import pandas as pd
from IPython.utils.text import indent

import requests
from bs4 import BeautifulSoup
from IPython.display import HTML

In [115]:
# Setting up 'requests' to make HTTPS requests properly takes some extra steps... we'll skip them for now.
requests.packages.urllib3.disable_warnings()

import warnings
warnings.filterwarnings("ignore")

In [116]:
snapshot_url = 'http://crimemap.finki.ukim.mk/data/mk'

In [117]:
snapshot = requests.get(snapshot_url)
snapshot

<Response [200]>

In [118]:
snapshot = requests.get(snapshot_url)
raw_html = snapshot.text
print(raw_html[:500])

<!DOCTYPE html>
<html>
    <head>
        <meta charset="utf-8">
        <link rel="stylesheet" href="../css/data.css" type="text/css">
        <link rel="stylesheet" href="../css/smoothness/jquery-ui-1.8.22.custom.css" type="text/css">
        <script type="text/javascript" src="../js/jquery-1.7.2.min.js"> </script>
        <script type="text/javascript" src="../js/jquery-ui-1.8.22.custom.min.js"></script>
        <script type="text/javascript"  src="../js/data.js"></script>
    </head>
    <bo


In [119]:
 soup = BeautifulSoup(raw_html, 'html.parser')

In [121]:
 soup

<!DOCTYPE html>

<html>
<head>
<meta charset="utf-8"/>
<link href="../css/data.css" rel="stylesheet" type="text/css"/>
<link href="../css/smoothness/jquery-ui-1.8.22.custom.css" rel="stylesheet" type="text/css"/>
<script src="../js/jquery-1.7.2.min.js" type="text/javascript"> </script>
<script src="../js/jquery-ui-1.8.22.custom.min.js" type="text/javascript"></script>
<script src="../js/data.js" type="text/javascript"></script>
</head>
<body>
<script src="../js/jquery.ui.datepicker-mk.js" type="text/javascript"></script><meta content="Crime Map Macedonia" property="og:title">
<meta content="cause" property="og:type"/>
<meta content="http://www.facebook.com/CrimeMapMacedonia" property="og:url"/>
<meta content="http://sphotos-b.ak.fbcdn.net/hphotos-ak-ash4/389494_418340018219724_87343145_n.png" property="og:image"/>
<meta content="Crime Map Macedonia" property="og:site_name"/>
<meta content="726043729" property="fb:admins"/>
<link href="../css/header.css" rel="stylesheet" type="text/css"

required class element names to be extracted:
  [id, shto, grad, adresa, datum, lat, lng, opis]

In [122]:
id_titles = soup.find_all('div', {'class': 'id'})
shto_titles = soup.find_all('div', {'class': 'shto'})
grad_titles = soup.find_all('div', {'class': 'grad'})
adresa_titles = soup.find_all('div', {'class': 'adresa'})
datum_titles = soup.find_all('div',{'class':'datum'})
datum_titles = np.array(datum_titles)[::2]
lat_titles = soup.find_all('div', {'class': 'lat'})
lng_titles = soup.find_all('div', {'class': 'lng'})
opis_titles = soup.find_all('textarea', {'class': 'opis'})


In [124]:
id = []
for i in range(0,len(id_titles)):
  id.append(id_titles[i].text)

shto = []
for i in range(0,len(shto_titles)):
  shto.append(shto_titles[i].text)

grad = []
for i in range(0,len(grad_titles)):
  grad.append(grad_titles[i].text)

adresa = []
for i in range(0,len(adresa_titles)):
  adresa.append(adresa_titles[i].text)

datum = []
for i in range(0,len(datum_titles)):
  datum.append(datum_titles[i])

lat = []
for i in range(0,len(lat_titles)):
  lat.append(lat_titles[i].text)

lng = []
for i in range(0,len(lng_titles)):
  lng.append(lng_titles[i].text)

opis = []
second_occurence_of_open_tag_symbol = str(opis_titles[0]).index('>')
third_occurence_of_open_tag_symbol = str(opis_titles[0]).index('<',2)
for i in range(0,len(opis_titles)):
  opis.append(str(opis_titles[i])[second_occurence_of_open_tag_symbol+1:third_occurence_of_open_tag_symbol+1].replace('<','').replace('>',''))

In [125]:
matrix = []
for i in range(0, len(shto)):
    matrix.append([id[i],shto[i],grad[i],datum[i],lat[i],lng[i],opis[i]])


df_new = pd.DataFrame(np.array(matrix),columns=['Id', 'Што', 'Град','Датум','Г.шир.', 'Г.дол', 'Опис'])

In [126]:
 df_new

,Id,Што,Град,Датум,Г.шир.,Г.дол,Опис
0,1,насилство,Скопјес. Горно Лисиче25.06.201124.06.201141.96...,[25.06.2011],41.9616704,21.5075581,"На 24.06.2011 година, СВР-Скопје, со кривична ..."
1,2,насилство,Штип25.06.201124.06.201141.7522.2На 24.06.201...,[25.06.2011],41.75,22.2,"На 24.06.2011 година М.С.(1962) од Штип, во С..."
2,3,кражба,Скопје25.06.201124.06.201142.00381221.452246Но...,[25.06.2011],42.003812,21.452246,"Ноќта помеѓу 23 и 24.06.2011 година, во перио..."
3,4,кражба,Скопјеул.„Црвена Општина“25.06.201124.06.20114...,[25.06.2011],41.9980329,21.4344496,"На 24.06.2011 година околу 15,40 часот во Ско..."
4,5,сообраќај,Куманово25.06.201124.06.201142.1333721.72584На...,[25.06.2011],42.13337,21.72584,На 24.06.2011 година СВР-Куманово до ОЈО-Кума...
5,6,друго,Тетовос.Жеровјане25.06.201124.06.201141.914574...,[25.06.2011],41.9145748,20.9484282,"На 24.06.2011 година во 23,00 часот во с.Жеро..."
6,7,друго,Штип27.06.201126.06.201141.7522.2СВР-Штип под...,[27.06.2011],41.75,22.2,СВР-Штип поднесе кривична пријава против Г.Д....
7,8,кражба,Штип27.06.201126.06.201141.7522.2Од страна на...,[27.06.2011],41.75,22.2,Од страна на СВР-Штип по превземени оператив...
8,9,друго,Радовиш27.06.201126.06.201141.636111122.466666...,[27.06.2011],41.6361111,22.4666667,ПС Радовиш поднесе кривична пријава против Г....
9,10,друго,Струмицас.Муртино27.06.201126.06.201141.415322...,[27.06.2011],41.4153,22.7267,"На 26.06.2011 година, во СВР Струмица е прија..."
